In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client openai pandas langchain


In [ ]:
import json
import time
from googleapiclient.discovery import build
from google.oauth2 import service_account

In [ ]:
from google.colab import auth
auth.authenticate_user()


**Go to Google Cloud Console:**
Google Cloud Console

**Create a Service Account:**

Navigate to IAM & Admin → Service Accounts.
Click Create Service Account.
**Grant Permissions:**

Assign the role: Editor or Owner.
Enable Google Sheets API under "APIs & Services".
**Download JSON Key:**

Go to the "Keys" tab.
Click "Add Key" → "Create New Key" → Select JSON.
Download the JSON file (e.g., my-google-sheets-key.json).


In [ ]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
import json

# Upload service account key file manually in Colab
from google.colab import files
uploaded = files.upload()

# Use the uploaded file (replace 'your_service_account.json' with actual file name)
SERVICE_ACCOUNT_FILE = list(uploaded.keys())[0]

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=["https://www.googleapis.com/auth/spreadsheets"]
)

service = build("sheets", "v4", credentials=creds)

Saving reference-baton-449912-i4-59268e0146a3.json to reference-baton-449912-i4-59268e0146a3 (1).json


In [ ]:
SPREADSHEET_ID = "1t7ApUNjqlA8gsXYSW8zyQbXzKfivRCm_4RInA8As3n0"
SHEET_NAME = "train (2)"  # Change if needed
STATE_FILE = "/content/last_row_state.json"  # Track last processed row
SHEET_ID = 1239881730 # Replace with the actual sheetId

In [ ]:
with open(STATE_FILE, "w") as f:
    json.dump({"last_row": 1}, f)

print("✅ Reset last processed row to 1. Now try running process_initial_data() again.")


✅ Reset last processed row to 1. Now try running process_initial_data() again.


In [ ]:
def get_last_processed_row():
    """Retrieve the last processed row from a local JSON file in Colab."""
    try:
        with open(STATE_FILE, "r") as f:
            data = json.load(f)
            last_row = data.get("last_row", 1)
            return max(1, last_row)  # Ensure at least 1
    except (FileNotFoundError, json.JSONDecodeError):
        return 1  # Default to 1 if the file does not exist


def set_last_processed_row(row):
    """Update the last processed row in a local JSON file."""
    with open(STATE_FILE, "w") as f:
        json.dump({"last_row": row}, f)

def get_total_rows():
    """Retrieve total number of rows in Column A, ignoring empty rows."""
    sheet = service.spreadsheets()

    # Force API refresh using a dynamic range
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=f"{SHEET_NAME}!A:A").execute()
    values = result.get("values", [])

    # Find the last row that contains data
    last_row = len(values) if values else 1
    return last_row

def update_google_sheet(start_row, end_row):
    """Apply Google Translate formula to new rows in Column D."""
    sheet = service.spreadsheets()

    if start_row > end_row:
        return  # No new rows

    print(f"✅ Applying formulas to rows {start_row} to {end_row}")

    formulas = [[f"=GOOGLETRANSLATE(C{i}, \"hi\", \"en\")"] for i in range(start_row, end_row + 1)]

    batch_size = 20  # Prevents API rate limits
    write_count = 0  # Track number of API requests
    for i in range(start_row, end_row + 1, batch_size):
        batch_end = min(i + batch_size - 1, end_row)
        body = {"values": formulas[i - start_row : batch_end - start_row + 1]}

        try:
            sheet.values().update(
                spreadsheetId=SPREADSHEET_ID,
                range=f"{SHEET_NAME}!D{i}:D{batch_end}",
                valueInputOption="USER_ENTERED",
                body=body,
            ).execute()
            print(f"✅ Updated rows {i} to {batch_end}.")
            write_count += 1

            # 🔹 Prevent quota limits: Wait after every 1000 requests
            if write_count % 50 == 0:
                print("⏳ Waiting 60 seconds to prevent quota limit...")
                time.sleep(60)

        except Exception as e:
            print(f"❌ Error updating rows {i} to {batch_end}: {str(e)}")
            break  # Stop processing if an error occurs

    set_last_processed_row(end_row)

def process_initial_data():
    """First run: Apply formulas to all existing data in batches to avoid API timeout."""
    total_rows = get_total_rows()
    last_processed = get_last_processed_row()

    # Ensure we start at row 2 (not the header)
    start_row = max(2, last_processed + 1)

    if start_row > total_rows:
        print("✅ All rows are already processed. Skipping...")
        return  # Nothing to process

    print(f"🔄 Applying formulas to rows {start_row} to {total_rows}...")

    batch_size = 1000  # Process 1000 rows at a time
    for i in range(start_row, total_rows + 1, batch_size):
        batch_end = min(i + batch_size - 1, total_rows)
        update_google_sheet(i, batch_end)

        print(f"⏳ Processed up to row {batch_end}. Waiting 60 seconds to prevent quota limit...")
        time.sleep(60)  # Wait after each large batch

    print("✅ Initial processing complete.")



# 🔹 New Rows Monitoring (Tracks from Last Row)
def monitor_new_rows():
    """Continuously monitor for new rows and process them from last row."""
    while True:
        try:
            last_processed = get_last_processed_row()
            total_rows = get_total_rows()

            if total_rows > last_processed:
                print(f"📌 New rows detected: {last_processed + 1} to {total_rows}")
                update_google_sheet(last_processed + 1, total_rows)
            else:
                print("✅ No new rows detected.")

            time.sleep(30)  # Check every 30 seconds

        except Exception as e:
            print(f"⚠️ Error in monitoring: {e}")
            time.sleep(60)  # Wait and retry if an error occurs

In [ ]:
def get_sheet_names():
    """Retrieve and print available sheet names in the Google Spreadsheet."""
    sheet_metadata = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
    sheets = sheet_metadata.get("sheets", [])
    print("Available sheets:")
    for s in sheets:
        print(s["properties"]["title"])


In [ ]:
get_sheet_names()

Available sheets:
train (2)


In [ ]:
process_initial_data()

🔄 Applying formulas to rows 2 to 93687...
✅ Applying formulas to rows 2 to 1001
✅ Updated rows 2 to 21.
✅ Updated rows 22 to 41.
✅ Updated rows 42 to 61.
✅ Updated rows 62 to 81.
✅ Updated rows 82 to 101.
✅ Updated rows 102 to 121.
✅ Updated rows 122 to 141.
✅ Updated rows 142 to 161.
✅ Updated rows 162 to 181.
✅ Updated rows 182 to 201.
✅ Updated rows 202 to 221.
✅ Updated rows 222 to 241.
✅ Updated rows 242 to 261.
✅ Updated rows 262 to 281.
✅ Updated rows 282 to 301.
✅ Updated rows 302 to 321.
✅ Updated rows 322 to 341.
✅ Updated rows 342 to 361.
✅ Updated rows 362 to 381.
✅ Updated rows 382 to 401.
✅ Updated rows 402 to 421.
✅ Updated rows 422 to 441.
✅ Updated rows 442 to 461.
✅ Updated rows 462 to 481.
✅ Updated rows 482 to 501.
✅ Updated rows 502 to 521.
✅ Updated rows 522 to 541.
✅ Updated rows 542 to 561.
✅ Updated rows 562 to 581.
✅ Updated rows 582 to 601.
✅ Updated rows 602 to 621.
✅ Updated rows 622 to 641.
✅ Updated rows 642 to 661.
✅ Updated rows 662 to 681.
✅ Updated ro

In [ ]:
print(get_full_form("TTYL"))

✅ Found full form: TTYL -> Talk To You Later
Talk To You Later
